In [125]:
import boto3

bedrock_client = boto3.client(service_name='bedrock-agent-runtime', region_name='us-west-2')

In [126]:
def retrieveAndGenerate(input_text, sourceType, model_id, region, document_s3_uri=None, data=None, identifier=None):
    model_arn = f'arn:aws:bedrock:{region}::foundation-model/{model_id}'

    if sourceType == "S3":
        return bedrock_client.retrieve_and_generate(
            input={'text': input_text},
            retrieveAndGenerateConfiguration={
                'type': 'EXTERNAL_SOURCES',
                'externalSourcesConfiguration': {
                    'modelArn': model_arn,
                    'sources': [
                        {
                            "sourceType": sourceType,
                            "s3Location": {
                                "uri": document_s3_uri  
                            }
                        }
                    ]
                }
            }
        )
        
    else:
        return bedrock_client.retrieve_and_generate(
            input={'text': input_text},
            retrieveAndGenerateConfiguration={
                'type': 'EXTERNAL_SOURCES',
                'externalSourcesConfiguration': {
                    'modelArn': model_arn,
                    'sources': [
                        {
                            "sourceType": sourceType,
                            "byteContent": {
                                "identifier": identifier,
                                "contentType": "text/plain",
                                "data": data  
                            }
                        }
                    ]
                }
            }
        )

In [127]:
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"    # Replace with your modelID
document_uri = "s3://rapports-annuels/Consommation de Base/Empire/2020-Empire-AR-English-SEDAR_compressed.pdf"  # Replace with your S3 URI

In [128]:
def my_chatbot(question):
    
    response = retrieveAndGenerate(
                                input_text=question,
                                sourceType="S3", 
                                model_id=model_id,
                                region="region_name",
                                document_s3_uri=document_uri
                              )

    return response

In [129]:
my_question = "revenue growth in 2020"
response = my_chatbot(question=my_question)
print(response['output']['text'])

ClientError: An error occurred (UnrecognizedClientException) when calling the RetrieveAndGenerate operation: The security token included in the request is invalid.

In [ ]:
import requests
from bs4 import BeautifulSoup
import boto3
import nltk
import matplotlib.pyplot as plt
import numpy as np


# Initialisation du client Amazon Comprehend en utilisant le fichier credentials par défaut
comprehend = boto3.client('comprehend', region_name='us-west-2')

def analyse(nom):

    for type in {"clients", nom}:
        # URL pour récupérer les tweets sur Bell Canada
        if type == nom:
            URL = f"https://twstalker.com/{nom}"
        else:
            URL = f"https://twstalker.com/search/{nom}"  # Utiliser un site de visualisation des tweets
        
        page = requests.get(URL)

        # Conversion de la page en format analysable avec BeautifulSoup
        soup = BeautifulSoup(page.content, "html.parser")

        # Extraction des tweets (ajuster le sélecteur en fonction de la structure réelle du site)
        tweets = []
        job_elements = soup.find_all("p")

        nb_tweets = 30

        # Limiter à 30 tweets et extraire le texte
        for i, job_element in enumerate(job_elements[:nb_tweets]):
            element = job_element.text
            tweets.append(element)

        tweets = tweets[1:]  # Retirer le premier élément si nécessaire
        print(f"Nombre de tweets extraits : {len(tweets)}")

        # Analyse de sentiment avec Amazon Comprehend
        positive_tweets = 0
        negative_tweets = 0
        neutral_tweets = 0
        mixed_tweets = 0

        sample_positive = None
        sample_negative = None
        sample_neutral = None
        sample_mixed = None

        def classify_sentiment_comprehend(tweet):
            response = comprehend.detect_sentiment(Text=tweet, LanguageCode='en')
            sentiment = response['Sentiment']
            return sentiment

        # Classification des tweets
        for tweet in tweets:
            sentiment = classify_sentiment_comprehend(tweet)
            if sentiment == 'POSITIVE':
                positive_tweets += 1
                if sample_positive is None:
                    sample_positive = tweet
            elif sentiment == 'NEGATIVE':
                negative_tweets += 1
                if sample_negative is None:
                    sample_negative = tweet
            elif sentiment == 'NEUTRAL':
                neutral_tweets += 1
                if sample_neutral is None:
                    sample_neutral = tweet
            elif sentiment == 'MIXED':
                mixed_tweets += 1
                if sample_mixed is None:
                    sample_mixed = tweet

        # Calcul du ratio
        total_tweets = len(tweets)
        if total_tweets > 0:
            positive_ratio = positive_tweets / total_tweets
            negative_ratio = negative_tweets / total_tweets
            neutral_ratio = neutral_tweets / total_tweets
            mixed_ratio = mixed_tweets / total_tweets
            
            print(f"Ratio de tweets positifs : {positive_ratio:.2%}")
            print(f"Ratio de tweets négatifs : {negative_ratio:.2%}")
            print(f"Ratio de tweets neutres : {neutral_ratio:.2%}")
            print(f"Ratio de tweets mixtes : {mixed_ratio:.2%}\n")
        else:
            print("Aucun tweet à analyser.")
            
        # Visualisation de la position de sentiment sur une barre
        cursor_position = positive_ratio - negative_ratio

        # Affichage de la barre horizontale avec dégradé
        fig, ax = plt.subplots(figsize=(10, 2))
        gradient = np.linspace(0, 1, 256).reshape(1, -1)
        gradient = np.vstack((gradient, gradient))

        ax.imshow(gradient, aspect='auto', cmap=plt.cm.RdYlGn, extent=[0, 1, 0, 1])

        # Ajouter le curseur indiquant la position de Bell sur la barre
        ax.annotate(f'{nom}\n(selon {type})', xy=((cursor_position + 1) / 2, 0.5), xytext=((cursor_position + 1) / 2, 0.7),
                    arrowprops=dict(facecolor='black', shrink=0.05), ha='center')

        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(f'Analyse de Sentiment des Tweets : Rouge (100% Négatif) à Vert (100% Positif)')
    plt.show()


def main():
    # Analyse de sentiment pour Bell Canada
    analyse("bellcanada")